In [96]:
import pandas as pd
import numpy as np
import tensorflow as tf

要predict的是each rally result

In [172]:
game_num = 2
path = 'set1_test.csv'
df_combined = pd.DataFrame()
for i in range(0, game_num):
    df = pd.read_csv(path)
    df.insert(0, 'Game', i)
    df = df.drop(index=df[df['Action'] == 'OP'].index.tolist())

    for idx, row in df.iterrows():
        if(row['Attempts'] == 1):
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_1'
        else:
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_0'
    
    df = df.drop(columns='Attempts')
    df_combined = pd.concat([df_combined, df])

df_combined = df_combined.reset_index(drop=True)
print(len(df_combined))
df_combined.head()

482


,Game,Team,Rally,No.,Space,Action,Errors,Score
0,0,IRI,1,2.0,Y,JS_0,NaN,NaN
1,0,POL,1,14.0,8,R_0,NaN,NaN
2,0,POL,1,19.0,2,G_0,NaN,NaN
3,0,POL,1,6.0,3,A_0,1.0,NaN
4,0,IRI,1,22.0,1,B_0,NaN,1.0


In [183]:
def process(df):
    '''
    one df represents one game
    split: space, action, player(?), error&score
    label: 發球方是否得分？
    '''
    rally_set = []
    rally_space_set = []
    rally_action_set = []
    rally_result_set = []
    cnt = 0
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        curr_team = df_combined['Team'][cnt]
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        shot_result_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []
        
        for _, df_shot in df_rally.iterrows():
            if(df_shot['Team'] != curr_team):
                shot_set.append(atk_sequence)
                curr_team = df_shot['Team']
                atk_sequence = []
            shot = df_shot.astype('float')
            tensor_shot = tf.convert_to_tensor(shot)
            atk_sequence.append(tensor_shot)
            cnt += 1
        shot_set.append(atk_sequence)
        rally_set.append(shot_set)
    
    return rally_set

In [184]:
df_combined = df_combined.replace({'IRI': 0, 'POL': 1})
df_dummy = pd.get_dummies(df_combined)

In [185]:
# rally_set[0][0][0][3:6]  # (rally, shot_sequence, attack_sequence, feature)

In [186]:
rally_set = process(df_dummy)

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 44)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(1, 34)
(1, 35)
(1, 36)
(1, 37)
(1, 38)
(1, 39)
(1, 40)
(1, 41)
(1, 42)
(1, 44)


In [176]:
start = 0
# print(rally_set[0][0][0][0:3])  # Game, Team, Rally (game跟rally不需要)
start += 3
print(rally_set[0][0][0][start:start+2])    # Errors, Attempt, Score (Attempt要合併到Action)
start += 2
print(tf.concat([rally_set[0][0][0][1:2], rally_set[0][0][0][start:start+len(df_combined.groupby('No.'))]], 0)) # Team & No.
# print(rally_set[0][0][0][start:start+len(df_combined.groupby('No.'))])
start += len(df_combined.groupby('No.'))
print(rally_set[0][0][0][start:start+len(df_combined.groupby('Space'))])
start += len(df_combined.groupby('Space'))
print(rally_set[0][0][0][start:start+len(df_combined.groupby('Action'))])

tf.Tensor([nan nan], shape=(2,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(23,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], shape=(14,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(10,), dtype=float64)


In [32]:
process(df_combined)

   Game Team  Rally   No. Space Action  Errors  Attempts  Score
0     0  IRI      1   2.0     Y     JS     NaN       NaN    NaN
1     0  POL      1  14.0     8      R     NaN       NaN    NaN
2     0  POL      1  19.0     2      G     NaN       NaN    NaN
3     0  POL      1   6.0     3      A     1.0       NaN    NaN
4     0  IRI      1  22.0     1      B     NaN       NaN    1.0
   Game Team  Rally   No. Space Action  Errors  Attempts  Score
5     0  IRI      2   2.0     Y     JS     NaN       NaN    NaN
6     0  POL      2  14.0     7      R     NaN       NaN    NaN
7     0  POL      2  19.0     1      G     NaN       NaN    NaN
8     0  POL      2   6.0     3      A     NaN       NaN    1.0
9     0  IRI      2   2.0     8      D     1.0       NaN    NaN
    Game Team  Rally   No. Space Action  Errors  Attempts  Score
10     0  POL      3    15     Y     JS     NaN       NaN    NaN
11     0  IRI      3   8.0     8      R     NaN       NaN    NaN
12     0  IRI      3  18.0     2     